In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from typing import List
from pydantic import BaseModel, Field

In [ ]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [ ]:
foo = User("Joe", 32, "ychag@example.com")

In [ ]:
foo.name

In [ ]:
foo = User("Joe", "bar", "ychag@example.com")

In [ ]:
foo.age

In [ ]:
class PUser(BaseModel):
        name: str
        age: int
        email: str

In [ ]:
foo_p = PUser(name="Joe", age=32, email="ychag@example.com")

In [ ]:
foo_p.name

In [ ]:
foo_p = PUser(name="Joe", age=1, email="ychag@example.com")

In [ ]:
class Class(BaseModel):
    students: List[PUser]

In [ ]:
obj = Class(
    students=[PUser(name="Joe", age=32, email="ychag@example.com")]
)

In [ ]:
obj

In [ ]:
# Pydantic to OpenAI function definition

In [ ]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at the airport"""
    airport_code : str = Field(description="The airport code to get weather for")

In [ ]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [ ]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [ ]:
weather_function

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
model = ChatOpenAI()

In [ ]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

In [ ]:
model_with_function = model.bind(functions=[weather_function])

In [ ]:
model_with_function.invoke("what is the weather in SF today?")

In [ ]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name": "WeatherSearch"})

In [ ]:
model_with_forced_function.invoke("what is the weather in SF today?")

In [ ]:
model_with_function.invoke("hi")

In [ ]:
model_with_forced_function.invoke("hi")

In [ ]:
# Using in a chain

In [ ]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [ ]:
chain = prompt | model_with_function

In [ ]:
chain.invoke({"input": "what is the weather in SF?"})